# Importing Libraries

In [2]:
import numpy as np
import pandas as pd 
import string
import warnings
warnings.filterwarnings('ignore')
import re

from textblob import TextBlob

import nltk
#nltk.download('punkt')
#nltk.download('wordnet')

import spacy


from nltk.corpus import stopwords

#Stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

#Lemmetization
from nltk.stem import WordNetLemmatizer
lemmetizer = WordNetLemmatizer()

from flask import Flask, redirect, url_for, request,render_template

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
#Words that we do not require after tokenization in the search term and the title column as well
unwanted_words = ['Email','Sample','Letter','Template','email','sample','letter','template','to','To']
#Punctuations
punctuations = '''()-[]{};:'"\,<>./?@#$%^&*_`~'''
#Stopwords
stop_words = stopwords.words('english')

# Importing dataset and Preprocessing

In [5]:
#Reading the data set and preprocessing
data = pd.read_excel('templates.xls')
data = data.dropna()
data = data.reset_index(drop=True)
#Replacing 'Letter' with 'Email' || Replacing '-' with ' ' in the cat column
for i in range (0,len(data)):
    data.iloc[i,0] = data.iloc[i,0].replace('Letter','Email')
    data.iloc[i,0] = data.iloc[i,0].replace('letter','Email')
    data.iloc[i,1] = data.iloc[i,1].replace('Letter','Email')
    data.iloc[i,1] = data.iloc[i,1].replace('letter','Email')
    data.iloc[i,0] = data.iloc[i,0].replace('-',' ')
data['Search_column'] = ''
data = data[["Category","Search_column", "Title", "Body"]]

#Removing the 'Download Related Samples...' from footer
for i in range (0,len(data)):
    txt = data.iloc[i,3]
    txt0 = txt.split('\nDownload ')[0]
    data.iloc[i,3] = txt0

#Removing address fields
for i in range (0,len(data)):
    list0 = data.Body[i].split('\n')
    list1 = []
    for string in list0:
        if len(string)> 50:
            list1.append(string)
    body = 'To: [Email id]\n\nSubject: [Subject]\n\n[Salutation]\n\n'
    for j in list1:
        body = body + j + '\n\n'
    body = body + '[Closure]\n\n'+'[Name/Signature]\n'+'[Designation]\n'+'[Organization]\n'
    body = body.replace('Letter','Email')
    body = body.replace('letter','email')
    #print(body)
    data.iloc[i,3] = body

#preprocessing the title column
for i in range (0,len(data)):
    tex = str(data.Title[i])
    tex_p = ''
    for char in tex:
        if (char not in punctuations):
            tex_p = tex_p+char
    #tex_p = re.sub(r'[^\w\s]','',tex)
    tex_p = tex_p.strip()
    tex_a = nltk.word_tokenize(tex_p)
    tex_s=[]
    for word in tex_a:
        if (word not in stop_words):
            tex_s.append(word)
    tex_l = []
    for word in tex_s:
        tex_l.append(lemmetizer.lemmatize(word))
    tex_f = []
    for word in tex_l:
        if word not in unwanted_words:
            tex_f.append(word)
        else:
            pass
    data.iloc[i,1] = tex_f

#Named Entity Recognition
for i in range(0,len(data)):
    s= data.Body[i]
    doc = nlp(s)
    newString = s
    for e in reversed(doc.ents): #reversed to not modify the offsets of other entities when substituting
        start = e.start_char
        end = start + len(e.text)
        newString = newString[:start] + '[' + e.label_ + ']' + newString[end:]
    data.Body[i] = newString
    
data.head()

Category                          Search_column  \
0  Acceptance Emails        [Elementary, School, Admission]   
1  Acceptance Emails                   [Academy, Admission]   
2  Acceptance Emails  [College, Admissions, Recommendation]   
3  Acceptance Emails                   [Decline, Admission]   
4  Acceptance Emails                  [Harvard, Acceptance]   

                                     Title  \
0        Elementary School Admission Email   
1                  Academy Admission Email   
2  College Admissions Recommendation Email   
3                  Decline Admission Email   
4                 Harvard Acceptance Email   

                                                Body  
0  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...  
1  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...  
2  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...  
3  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...  
4  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...

# EDA

In [6]:
data.shape

(1574, 4)

In [7]:
data.columns

Index(['Category', 'Search_column', 'Title', 'Body'], dtype='object')

The dataset consists of 4 columns namely 'Category', 'Search_column', 'Title' and 'Body'.
<br>Dataset altogether consists of 1574 Email templates.

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1574 entries, 0 to 1573
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Category       1574 non-null   object
 1   Search_column  1574 non-null   object
 2   Title          1574 non-null   object
 3   Body           1574 non-null   object
dtypes: object(4)
memory usage: 49.3+ KB


As we can see that there are no null values.

In [9]:
print('Length: '+str(len(data.Category.unique()))+'\n')
data.Category.unique()

Length: 53



array(['Acceptance Emails', 'Admission Emails', 'Announcement Emails',
       'Application Emails', 'Appreciation Emails',
       'Authorization Emails', 'Banking Emails', 'Birthday Email',
       'Business Emails', 'Cancellation Email', 'Certification Email',
       'Claim Email', 'Complaint Emails', 'Condolence Emails',
       'Confirmation Email', 'Congratulation Emails', 'Consumer Emails',
       'Customer Emails', 'Decline Email', 'Emails', 'Employment Emails',
       'Finance Emails', 'Formal Email', 'General Emails',
       'Government Emails', 'Grievance Emails', 'Holiday Emails',
       'Inquiry Email', 'Internship Email', 'Interview Email',
       'Introduction Emails', 'Invitation Emails', 'Job Emails',
       'Leave Emails', 'Marketing Emails', 'Order Email',
       'Permission Email', 'Personal Emails', 'Recommendation Emails',
       'Reference Emails', 'Request Emails', 'Resignation Emails',
       'Retirement Emails', 'Sales & Marketing Emails', 'Sales Emails',
       '

As we can see that there are 53 Categories as shown above.

The title column shows the title of each Email

The Search_Column consists of the tokens from the title column for matching the keywords.

The Body column consists of templates of the email.

# Search Term Input and Processing

In [10]:
g_text = input('Search Email Templates:\n\n')

Search Email Templates:

Sick Leave EMail Template


In [11]:
text=''
text = re.sub(r'[^\w\s]','',g_text)
text

'Sick Leave EMail Template'

In [12]:
#text=''
#for char in g_text:
#    if (char not in punctuations):
#        text = text+char
#text

In [13]:
text = text.strip()
text = text.title()
text

'Sick Leave Email Template'

In [14]:
text_a = nltk.word_tokenize(text)
text_a

['Sick', 'Leave', 'Email', 'Template']

In [15]:
text_s=[]
for word in text_a:
    if (word not in stop_words):
        text_s.append(word)
        
text_s

['Sick', 'Leave', 'Email', 'Template']

In [16]:
text_l = []
for word in text_s:
#    print(lemmetizer.lemmatize(word))
    if word == 'boss':
        text_l.append(word)
    else: 
        text_l.append(lemmetizer.lemmatize(word))
text_l

['Sick', 'Leave', 'Email', 'Template']

In [17]:
text_f = []
for word in text_l:
    if word not in unwanted_words:
        text_f.append(word)
    else:
        pass
text_f

['Sick', 'Leave']

## Searching for keywords in Search_column

In [18]:
data.head()

Category                          Search_column  \
0  Acceptance Emails        [Elementary, School, Admission]   
1  Acceptance Emails                   [Academy, Admission]   
2  Acceptance Emails  [College, Admissions, Recommendation]   
3  Acceptance Emails                   [Decline, Admission]   
4  Acceptance Emails                  [Harvard, Acceptance]   

                                     Title  \
0        Elementary School Admission Email   
1                  Academy Admission Email   
2  College Admissions Recommendation Email   
3                  Decline Admission Email   
4                 Harvard Acceptance Email   

                                                Body  
0  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...  
1  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...  
2  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...  
3  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...  
4  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...

In [19]:
#Searching for keywords in the search column
result_in = []
for word in text_f:
    for i in range (0,len(data)):
        for j in range (0,len(data.iloc[i,1])):
            if data.iloc[i,1][j] not in text_f:
                pass
            else:
                result_in.append(int(i))
result_in.sort(reverse=False)
result_in

[129,
 129,
 257,
 257,
 978,
 978,
 979,
 979,
 980,
 980,
 981,
 981,
 982,
 982,
 983,
 983,
 984,
 984,
 985,
 985,
 986,
 986,
 987,
 987,
 989,
 989,
 990,
 990,
 991,
 991,
 995,
 995,
 997,
 997,
 998,
 998,
 999,
 999,
 1000,
 1000,
 1001,
 1001,
 1003,
 1003,
 1004,
 1004,
 1203,
 1203,
 1486,
 1486]

In [20]:
x = data.iloc[result_in]
x['priority'] = np.NaN
x.head()

Category                     Search_column  \
129  Application Emails    [Marriage, Leave, Application]   
129  Application Emails    [Marriage, Leave, Application]   
257     Business Emails  [Short, Request, Leave, Absence]   
257     Business Emails  [Short, Request, Leave, Absence]   
978        Leave Emails                  [Leave, Teacher]   

                                  Title  \
129          Marriage Leave Application   
129          Marriage Leave Application   
257  Short Request for Leave of Absence   
257  Short Request for Leave of Absence   
978              Leave Email to Teacher   

                                                  Body  priority  
129  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       NaN  
129  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       NaN  
257  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       NaN  
257  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       NaN  
978  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       NaN

In [21]:
p = x.index.value_counts()
p

1203    2
987     2
1486    2
978     2
979     2
980     2
981     2
982     2
983     2
984     2
985     2
986     2
989     2
1004    2
990     2
991     2
995     2
997     2
998     2
999     2
1000    2
1001    2
257     2
1003    2
129     2
dtype: int64

In [22]:
x = x.reset_index()
p = p.reset_index()
p = p.rename(columns={0:'priority'},inplace=False)

In [23]:
x.head()

index            Category                     Search_column  \
0    129  Application Emails    [Marriage, Leave, Application]   
1    129  Application Emails    [Marriage, Leave, Application]   
2    257     Business Emails  [Short, Request, Leave, Absence]   
3    257     Business Emails  [Short, Request, Leave, Absence]   
4    978        Leave Emails                  [Leave, Teacher]   

                                Title  \
0          Marriage Leave Application   
1          Marriage Leave Application   
2  Short Request for Leave of Absence   
3  Short Request for Leave of Absence   
4              Leave Email to Teacher   

                                                Body  priority  
0  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       NaN  
1  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       NaN  
2  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       NaN  
3  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       NaN  
4  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       NaN

In [24]:
p.head()

index  priority
0   1203         2
1    987         2
2   1486         2
3    978         2
4    979         2

In [25]:
for i in range(0,len(x)):
    ind = x['index'][i] 
    for j in range(0,len(p)):
        if p['index'][j] == ind:
            pr = p['priority'][j]
        else:
            pass
    x['priority'][i] = pr
x.head()

index            Category                     Search_column  \
0    129  Application Emails    [Marriage, Leave, Application]   
1    129  Application Emails    [Marriage, Leave, Application]   
2    257     Business Emails  [Short, Request, Leave, Absence]   
3    257     Business Emails  [Short, Request, Leave, Absence]   
4    978        Leave Emails                  [Leave, Teacher]   

                                Title  \
0          Marriage Leave Application   
1          Marriage Leave Application   
2  Short Request for Leave of Absence   
3  Short Request for Leave of Absence   
4              Leave Email to Teacher   

                                                Body  priority  
0  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       2.0  
1  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       2.0  
2  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       2.0  
3  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       2.0  
4  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...       2.0

# Results


In [26]:
x = x.sort_values(by=['priority'],ascending=False)
x = x.drop_duplicates(subset=['index'])
x = x.drop(['index','Search_column','priority'],axis=1)
x = x.reset_index(drop=True)
x.head()

Category                           Title  \
0  Application Emails      Marriage Leave Application   
1        Leave Emails      Marriage Leave Application   
2        Leave Emails              Leave Email format   
3        Leave Emails  Employee Maternity Leave Email   
4        Leave Emails         Leave application Email   

                                                Body  
0  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...  
1  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...  
2  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...  
3  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...  
4  To: [Email id]\n\nSubject: [Subject]\n\n[Salut...

In [27]:
print('Category: ',x.Category[0],'\n')
print('Title: ',x.Title[0])
print('____________________________________________________________________________________________________________________________\n')
print(x.Body[0])

Category:  Application Emails 

Title:  Marriage Leave Application
____________________________________________________________________________________________________________________________

To: [Email id]

Subject: [Subject]

[Salutation]

I am [PERSON] and I work as the Chief Engineer with your team. I take immense pleasure in inviting you to my marriage with [PERSON] on the [ORDINAL] of May. I will be sending you the personal invitation in due course of time. The actual reason is a request to you to grant me a leave of absence for [DATE] starting from the [ORDINAL] of May. Kindly treat my absence as informed leave and oblige.

[Closure]

[Name/Signature]
[Designation]
[Organization]



### Just for future reference

In [28]:
df = pd.DataFrame(data.Category.value_counts())
df = df.reset_index()
df = df.rename(columns={'index':'Category','Category':'N_Letters'})
df

Category  N_Letters
0            Business Emails         70
1            Personal Emails         70
2            Customer Emails         70
3             General Emails         64
4   Sales & Marketing Emails         60
5             Finance Emails         51
6      Congratulation Emails         50
7           Thank You Emails         50
8           Complaint Emails         50
9          Condolence Emails         49
10            Request Emails         43
11        Application Emails         43
12       Announcement Emails         42
13       Appreciation Emails         42
14         Acceptance Emails         40
15             Uncategorized         36
16           Consumer Emails         35
17        Resignation Emails         35
18            Banking Emails         34
19         Employment Emails         33
20         Government Emails         33
21     Recommendation Emails         32
22              Sales Emails         32
23         Invitation Emails         31
24          Reference Emails         31
25       Introduction Emails         29
26            Holiday Emails         29
27        Termination Emails         28
28          Marketing Emails         27
29              Leave Emails         27
30         Retirement Emails         25
31           Supplier Emails         25
32          Statement Emails         22
33        Scholarship Emails         22
34           Interview Email         21
35          Grievance Emails         20
36          Admission Emails         19
37                    Emails         15
38              Formal Email         14
39      Authorization Emails         12
40               Claim Email         11
41        Cancellation Email         11
42            Welcome Emails         10
43       Certification Email         10
44          Internship Email         10
45             Inquiry Email         10
46        Confirmation Email         10
47          Permission Email          9
48            Birthday Email          8
49             Decline Email          8
50               Sorry Email          7
51               Order Email          5
52                Job Emails          4

# Deployment

In [166]:
app = Flask(__name__)

#app.static_folder = 'static'

@app.route('/')
def home_page():
    cats = df.Category.to_list()
    return render_template('index.html',cat_list = cats)

@app.route('/category',methods = ['POST','GET'])
def category():
    cats = df.Category.to_list()
    if request.method == 'POST':
        s = str(request.form['cat'])
        b = data[data.Category == s]
        title = b.Title.to_list()
    return render_template('cat_title.html',title_list=title,cat_list = cats)

@app.route('/category/title',methods = ['POST','GET'])
def cat_title():
    if request.method == 'POST':
        s = str(request.form['title'])
        b = data[data.Title == s]
        b = b.reset_index(drop=True)
        letter = 'Category: '+str(b.Category[0])+'\nTitle: '+str(b.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(b.Body[0])
        letter = letter.split('\n')
        cats = df.Category.to_list()
        c = data[data.Category == b.Category[0]]
        title = c.Title.to_list()
    return render_template('print.html',title_list = title,cat_list = cats,letter=letter)


        

@app.route('/search_results',methods = ['POST','GET'])
def search():
    if request.method == 'POST':
        s = str(request.form['keywords'])
        text=''
        text = re.sub(r'[^\w\s]','',s)
        text = text.strip()
        text = text.title()
        text_a = nltk.word_tokenize(text)
        text_s=[]
        for word in text_a:
            if (word not in stop_words):
                text_s.append(word)
        text_l = []
        for word in text_s:
            if word == 'boss':
                text_l.append(word)
            else: 
                text_l.append(lemmetizer.lemmatize(word))
        text_f = []
        for word in text_l:
            if word not in unwanted_words:
                text_f.append(word)
            else:
                pass
        #Searching for keywords in the search column
        result_in = []
        for word in text_f:
            for i in range (0,len(data)):
                for j in range (0,len(data.iloc[i,1])):
                    if data.iloc[i,1][j] not in text_f:
                        pass
                    else:
                        result_in.append(int(i))
        result_in.sort(reverse=False)
        x = data.iloc[result_in]
        x['priority'] = np.NaN
        p = x.index.value_counts()
        x = x.reset_index()
        p = p.reset_index()
        p = p.rename(columns={0:'priority'},inplace=False)
        for i in range(0,len(x)):
            ind = x['index'][i] 
            for j in range(0,len(p)):
                if p['index'][j] == ind:
                    pr = p['priority'][j]
                else:
                    pass
            x['priority'][i] = pr
        x = x.sort_values(by=['priority'],ascending=False)
        x = x.drop_duplicates(subset=['index'])
        x = x.drop(['index','Search_column','priority'],axis=1)
        x = x.reset_index(drop=True)
        #letter = 'Category: '+str(x.Category[0])+'\nTitle: '+str(x.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(x.Body[0])
        #letter = letter.split('\n')
        cats = df.Category.to_list()
        title_list = x.Title.to_list()
    return render_template('keywordsearch.html',cat_list = cats,title_list=title_list)

@app.route('/search_results/print',methods = ['POST','GET'])
def print_search_results():
    if request.method == 'POST':
        s = str(request.form['title'])
        b = data[data.Title == s]
        b = b.reset_index(drop=True)
        letter = 'Category: '+str(b.Category[0])+'\nTitle: '+str(b.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(b.Body[0])
        letter = letter.split('\n')
        cats = df.Category.to_list()
        c = data[data.Category == b.Category[0]]
        title = c.Title.to_list()
    return render_template('searchresultprint.html',letter = letter,cat_list = cats,title_list=title)

In [167]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Sep/2021 18:41:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2021 18:41:20] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2021 18:41:25] "POST /category/title HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2021 18:42:29] "POST /search_results HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2021 18:42:35] "POST /search_results/print HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2021 18:42:41] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2021 18:42:46] "POST /category/title HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2021 18:43:35] "POST /search_results HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2021 18:43:50] "POST /search_results/print HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2021 18:44:15] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2021 18:44:33] "POST /category HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2021 18:44:42] "POST /category/title HTTP/1.1" 200 -


In [136]:
x.columns[1]

'Title'

In [135]:
x.Title.to_list()

['Marriage Leave Application',
 'Marriage Leave Application',
 'Leave Email format',
 'Employee Maternity Leave Email',
 'Leave application Email',
 'Maternity Leave Email',
 'Sample Leave Email',
 'School Leave Application',
 'Leave Application E-mail',
 'Parental Leave Email',
 'Sick leave Email',
 'Maternity Leave Request Email',
 'Leave Acceptance Thank You Email',
 'Annual Leave Email',
 'Leave Email To Boss',
 'Short Request for Leave of Absence',
 'Leave Email to Teacher',
 'Half Day Leave Email',
 'Military Leave Email',
 'Leave Extension Email',
 'College Leave Email',
 'Effective Leave Application Email',
 'Leave Email For Condolence',
 'Leave Email For Marriage',
 'Leave Email to Manager']

In [ ]:
text=''
        text = re.sub(r'[^\w\s]','',s)
        text = text.strip()
        text = text.title()
        text_a = nltk.word_tokenize(text)
        text_s=[]
        for word in text_a:
            if (word not in stop_words):
                text_s.append(word)
        text_l = []
        for word in text_s:
            if word == 'boss':
                text_l.append(word)
            else: 
                text_l.append(lemmetizer.lemmatize(word))
        text_f = []
        for word in text_l:
            if word not in unwanted_words:
                text_f.append(word)
            else:
                pass
        #Searching for keywords in the search column
        result_in = []
        for word in text_f:
            for i in range (0,len(data)):
                for j in range (0,len(data.iloc[i,1])):
                    if data.iloc[i,1][j] not in text_f:
                        pass
                    else:
                        result_in.append(int(i))
        result_in.sort(reverse=False)
        x = data.iloc[result_in]
        x['priority'] = np.NaN
        p = x.index.value_counts()
        x = x.reset_index()
        p = p.reset_index()
        p = p.rename(columns={0:'priority'},inplace=False)
        for i in range(0,len(x)):
            ind = x['index'][i] 
            for j in range(0,len(p)):
                if p['index'][j] == ind:
                    pr = p['priority'][j]
                else:
                    pass
            x['priority'][i] = pr
        x = x.sort_values(by=['priority'],ascending=False)
        x = x.drop_duplicates(subset=['index'])
        x = x.drop(['index','Search_column','priority'],axis=1)
        x = x.reset_index(drop=True)
        letter = 'Category: '+str(x.Category[0])+'\nTitle: '+str(x.Title[0])+'\n____________________________________________________________________________________________________________\n'+str(x.Body[0])
        letter = letter.split('\n')
        cats = df.Category.to_list()
        title_list = x.Title.to_list()